<a href="https://colab.research.google.com/github/Alan-Cheong/IEEE_QW_2020/blob/master/Real_Estate_Procurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cv2
import numpy as np
from typing import Dict, List, Optional, Tuple
import pandas as pd
from dataclasses import dataclass
from enum import Enum
import json
import requests
from pathlib import Path

# Mock OpenAI API call (replace with actual OpenAI client)
def analyze_with_multimodal_ai(video_frames: List[np.ndarray], description: str) -> Dict:
    """
    Simulate OpenAI GPT-4V analysis of video frames and description
    In reality, you'd use the OpenAI client to send frames and get structured output
    """
    # This would be your actual OpenAI API call
    mock_response = {
        "square_footage": 2500,
        "meeting_rooms": 4,
        "open_workspace": True,
        "parking_spots": 20,
        "natural_light": "Excellent",
        "accessibility": "Full ADA compliance",
        "kitchen_facilities": "Full kitchen with dishwasher",
        "internet_infrastructure": "Fiber optic, Cat6 throughout",
        "hvac_system": "Central air with individual zone control",
        "security_features": ["24/7 security", "Key card access", "Security cameras"],
        "estimated_condition": "Excellent",
        "nearby_amenities": ["Restaurants", "Public transport", "Parking garage"]
    }
    return mock_response

@dataclass
class OfficeRequirement:
    """Define company requirements with priorities"""
    name: str
    required: bool
    weight: float  # 1-10 importance scale
    min_value: Optional[float] = None
    max_value: Optional[float] = None
    acceptable_values: Optional[List[str]] = None

class ComplianceLevel(Enum):
    EXCEEDS = "Exceeds"
    MEETS = "Meets"
    PARTIAL = "Partial"
    FAILS = "Fails"

class OfficeSpaceAnalyzer:
    """Main class for analyzing office spaces against company requirements"""

    def __init__(self, company_requirements: List[OfficeRequirement]):
        self.requirements = {req.name: req for req in company_requirements}

    def extract_video_frames(self, video_path: str, frame_interval: int = 30) -> List[np.ndarray]:
        """Extract key frames from video for AI analysis"""
        cap = cv2.VideoCapture(video_path)
        frames = []
        frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % frame_interval == 0:
                # Resize for efficiency
                frame = cv2.resize(frame, (800, 600))
                frames.append(frame)

            frame_count += 1

        cap.release()
        return frames[:10]  # Limit to 10 key frames

    def detect_office_features(self, frames: List[np.ndarray]) -> Dict:
        """Use computer vision to detect basic office features"""
        # Simplified feature detection - in reality you'd use more sophisticated models
        features = {
            "has_windows": False,
            "meeting_rooms_detected": 0,
            "open_areas": 0,
            "average_brightness": 0
        }

        if not frames:  # Add check for empty frames
            print("Warning: No frames extracted from video. Skipping feature detection.")
            return features

        for frame in frames:
            # Simple brightness analysis
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            brightness = np.mean(gray)
            features["average_brightness"] += brightness

            # Simple edge detection for rooms/spaces
            edges = cv2.Canny(gray, 50, 150)
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Rough estimation of distinct areas
            large_contours = [c for c in contours if cv2.contourArea(c) > 1000]
            features["open_areas"] += len(large_contours)


        features["average_brightness"] /= len(frames)
        features["has_windows"] = features["average_brightness"] > 100

        return features

    def analyze_property(self, video_path: str, description: str,
                        property_id: str) -> Dict:
        """Complete analysis of a property"""

        # Extract video frames
        frames = self.extract_video_frames(video_path)

        # Computer vision analysis
        cv_features = self.detect_office_features(frames)

        # AI multimodal analysis
        ai_specs = analyze_with_multimodal_ai(frames, description)

        # Combine all data
        property_data = {
            "property_id": property_id,
            "ai_extracted_specs": ai_specs,
            "cv_features": cv_features,
            "description": description
        }

        # Score against requirements
        compliance_score = self.score_property(property_data)
        property_data["compliance_score"] = compliance_score

        return property_data

    def score_property(self, property_data: Dict) -> Dict:
        """Score property against company requirements"""
        scores = {}
        total_weighted_score = 0
        total_weight = 0

        specs = property_data["ai_extracted_specs"]

        for req_name, requirement in self.requirements.items():
            score_info = {
                "compliance": ComplianceLevel.FAILS,
                "score": 0,
                "notes": ""
            }

            # Check if the property has this specification
            property_value = specs.get(req_name.lower().replace(" ", "_"))

            if property_value is None:
                score_info["notes"] = "Information not available"
                scores[req_name] = score_info
                continue

            # Score based on requirement type
            if requirement.min_value is not None:
                if isinstance(property_value, (int, float)):
                    if property_value >= requirement.min_value:
                        score_info["compliance"] = ComplianceLevel.MEETS
                        score_info["score"] = min(10, (property_value / requirement.min_value) * 8)
                    else:
                        score_info["compliance"] = ComplianceLevel.FAILS
                        score_info["score"] = (property_value / requirement.min_value) * 5

            elif requirement.acceptable_values:
                if property_value in requirement.acceptable_values:
                    score_info["compliance"] = ComplianceLevel.MEETS
                    score_info["score"] = 8
                else:
                    score_info["compliance"] = ComplianceLevel.PARTIAL
                    score_info["score"] = 4

            else:
                # Boolean or qualitative assessment
                if property_value:
                    score_info["compliance"] = ComplianceLevel.MEETS
                    score_info["score"] = 8

            # Apply weighting
            weighted_score = score_info["score"] * requirement.weight
            total_weighted_score += weighted_score
            total_weight += requirement.weight

            scores[req_name] = score_info

        # Calculate overall score
        overall_score = (total_weighted_score / total_weight) if total_weight > 0 else 0

        return {
            "overall_score": round(overall_score, 2),
            "detailed_scores": scores,
            "total_requirements": len(self.requirements),
            "requirements_met": sum(1 for s in scores.values()
                                  if s["compliance"] in [ComplianceLevel.MEETS, ComplianceLevel.EXCEEDS])
        }

    def generate_comparison_report(self, properties: List[Dict]) -> pd.DataFrame:
        """Generate a comparison report for all analyzed properties"""
        comparison_data = []

        for prop in properties:
            row = {
                "Property ID": prop["property_id"],
                "Overall Score": prop["compliance_score"]["overall_score"],
                "Requirements Met": prop["compliance_score"]["requirements_met"],
                "Square Footage": prop["ai_extracted_specs"].get("square_footage", "N/A"),
                "Meeting Rooms": prop["ai_extracted_specs"].get("meeting_rooms", "N/A"),
                "Parking": prop["ai_extracted_specs"].get("parking_spots", "N/A"),
                "Natural Light": prop["ai_extracted_specs"].get("natural_light", "N/A")
            }
            comparison_data.append(row)

        df = pd.DataFrame(comparison_data)
        return df.sort_values("Overall Score", ascending=False)

# Example usage
def main():
    # Define your company's office requirements
    company_requirements = [
        OfficeRequirement("Square Footage", required=True, weight=9, min_value=2000),
        OfficeRequirement("Meeting Rooms", required=True, weight=8, min_value=3),
        OfficeRequirement("Parking Spots", required=True, weight=7, min_value=15),
        OfficeRequirement("Natural Light", required=False, weight=6,
                         acceptable_values=["Good", "Excellent"]),
        OfficeRequirement("Kitchen Facilities", required=True, weight=5),
        OfficeRequirement("Accessibility", required=True, weight=10,
                         acceptable_values=["Full ADA compliance"]),
        OfficeRequirement("Internet Infrastructure", required=True, weight=8)
    ]

    # Initialize analyzer
    analyzer = OfficeSpaceAnalyzer(company_requirements)

    # Analyze properties (in reality, you'd have actual video files)
    properties = []

    # Mock property analysis
    for i in range(3):
        property_data = analyzer.analyze_property(
            video_path=f"mock_video_{i}.mp4",  # Would be actual video path
            description=f"Modern office space {i+1} with great amenities",
            property_id=f"PROP_{i+1:03d}"
        )
        properties.append(property_data)

    # Generate comparison report
    comparison_df = analyzer.generate_comparison_report(properties)
    print("Office Space Comparison Report:")
    print("=" * 50)
    print(comparison_df.to_string(index=False))

    # Show detailed analysis for top property
    best_property = properties[0]  # Assuming sorted by score
    print(f"\nDetailed Analysis for {best_property['property_id']}:")
    print("=" * 30)

    for req_name, score_info in best_property["compliance_score"]["detailed_scores"].items():
        print(f"{req_name}: {score_info['compliance'].value} (Score: {score_info['score']:.1f})")
        if score_info["notes"]:
            print(f"  Notes: {score_info['notes']}")

if __name__ == "__main__":
    main()

Office Space Comparison Report:
Property ID  Overall Score  Requirements Met  Square Footage  Meeting Rooms  Parking Natural Light
   PROP_001           8.91                 7            2500              4       20     Excellent
   PROP_002           8.91                 7            2500              4       20     Excellent
   PROP_003           8.91                 7            2500              4       20     Excellent

Detailed Analysis for PROP_001:
Square Footage: Meets (Score: 10.0)
Meeting Rooms: Meets (Score: 10.0)
Parking Spots: Meets (Score: 10.0)
Natural Light: Meets (Score: 8.0)
Kitchen Facilities: Meets (Score: 8.0)
Accessibility: Meets (Score: 8.0)
Internet Infrastructure: Meets (Score: 8.0)
